In [162]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
import torch
import torch.nn.functional as F

In [164]:
def test_near(a,b):
    return torch.allclose(a, b, rtol=1e-3, atol=1e-5)

In [165]:
# Function for logsoftmax and nll (negative log likelihood) 
# Function for Cross Entropy Loss

Softmax: $\sigma(\mathbf{z})_j = \frac{e^{z_j}}{\sum_{k=1}^K e^{z_k}}$ &nbsp;&nbsp; for ''j'' = 1, …, ''K''.

In [166]:
# softmax

inp = torch.FloatTensor([0.2, 0.1, 0.5, 0.1, 0.15]).unsqueeze(0)
target = torch.LongTensor([2])

smax = inp.exp() / inp.exp().sum()
smax

tensor([[0.1957, 0.1770, 0.2641, 0.1770, 0.1861]])

In [167]:
inp.shape

torch.Size([1, 5])

In [168]:
torch_smax = F.softmax(inp, dim=1)

assert test_near(smax, torch_smax)

$NLL = -\log(p_x)$

In [169]:
# logsoftmax

$\log_b(\tfrac{x}{y})=\log_b(x)-\log_b(y)$

In [170]:
lsmax = inp.exp().log() - inp.exp().sum().log()
lsmax

tensor([[-1.6313, -1.7313, -1.3313, -1.7313, -1.6813]])

$\log \left ( \sum_{j=1}^{n} e^{x_{j}} \right ) = \log \left ( e^{a} \sum_{j=1}^{n} e^{x_{j}-a} \right ) = a + \log \left ( \sum_{j=1}^{n} e^{x_{j}-a} \right )$

In [171]:
logexpsum = inp.exp().sum().log()

logexpsum_trick = inp.max() + (inp-inp.max()).exp().sum().log()

assert logexpsum == logexpsum_trick

In [172]:
def logsoftmax(inp):
    return inp.exp().log() - (inp.max() + (inp-inp.max()).exp().sum().log())
    
lsmax = logsoftmax(inp)

In [173]:
torch_lsmax = F.log_softmax(inp, dim=1)

assert test_near(lsmax, torch_lsmax)

In [174]:
# negative log likelihood

In [175]:
-lsmax[0, target]

tensor([1.3313])

In [176]:
def nll(log_softmax, target):
    return -log_softmax[range(target.shape[0]), target]

In [177]:
loss = nll(logsoftmax(inp), target)
loss

tensor([1.3313])

In [179]:
nn_loss = F.cross_entropy(inp, target)

assert loss == nn_loss

In [180]:
# Fin